In [11]:
import csv
import os
import sys
import re
import pandas as pd
import nltk
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [5]:
metrics = ['cbo','wmc','rfc','lcom','nom','nopm','nosm','nof','nopf','nosf','nosi','loc']

dirs = os.listdir("../data/Archive/")
dirs = [d for d in dirs if d != '.DS_Store']

In [17]:
samples =  {'source_code': [],
            'type': [],
            'cbo': [],
            'wmc': [],
            'rfc': [],
            'lcom': [],
            'nom': [],
            'nopm': [],
            'nosm': [],
            'nof': [],
            'nopf': [],
            'nosf': [],
            'nosi': [],
            'loc': [],
            "commits": [],
            "linesAdded": [],
            "linesDeleted": [],
            "authors": [],
            "minorAuthors": [],
            "majorAuthors": [],
            "authorOwnership": [],
            'target': []
            }


metrics2 = ["commits", "linesAdded", "linesDeleted", "authors", "minorAuthors", "majorAuthors", "authorOwnership"]
def get_values(row):
    metrics_values = {}
    for m in metrics2:
        metrics_values[m] = row[m]
    return metrics_values


def tokenize_code(code):
    return ' '.join(nltk.tokenize.wordpunct_tokenize(code))

def process_content(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
        source_code = []
        for line in lines:
            # filter comments
            if not re.match("\s*\/\/\s*isComment", line):
                source_code.append(tokenize_code(line))
    text = ' '.join(source_code)
    return text

In [18]:
for d in dirs:
    look_up = {}
    with open('../data/Archive/{}/process.csv'.format(d), 'r') as pro_file:
        pro_csv = csv.DictReader(pro_file, delimiter=';')
        for row in pro_csv:
            look_up["{}_{}".format(row["commit"], row["file"])] = get_values(row)

    with open('../data/Archive/{}/yes-refactoring.csv'.format(d), 'r') as pos_file:
        pos_csv = csv.DictReader(pos_file, delimiter=';')
        for row in pos_csv:
            
            samples['source_code'].append(process_content("../data/Archive/{}/storage/{}/before/{}".format(d,row['before'],row['path'])))
            samples['type'].append(row['type'])
            samples['target'].append(row['refactoring'])
            for m in metrics:
                samples[m].append(row[m])
                
            metrics2 = look_up["{}_{}".format(row["before"], row["path"])]
            for k, v in metrics2.items():
                samples[k].append(v)
            
    with open('../data/Archive/{}/no-refactoring.csv'.format(d), 'r') as neg_file:
        neg_csv = csv.DictReader(neg_file, delimiter=';')
        for row in neg_csv:
            samples['source_code'].append(process_content("../data/Archive/{}/storage/{}/not-refactored/{}".format(d,row['commit'],row['path'])))
            samples['type'].append(row['type'])
            samples['target'].append("NO_REFACTORING")
            for m in metrics:
                samples[m].append(row[m])
                
            metrics2 = look_up["{}_{}".format(row["commit"], row["path"])]
            for k, v in metrics2.items():
                samples[k].append(v)

In [19]:
df = pd.DataFrame.from_dict(samples)
df.head()

,source_code,type,cbo,wmc,rfc,lcom,nom,nopm,nosm,nof,...,nosi,loc,commits,linesAdded,linesDeleted,authors,minorAuthors,majorAuthors,authorOwnership,target
0,/* * Licensed to the Apache Software Foundatio...,class,10,63,18,4,20,14,1,11,...,9,400,18,20,18,3,0,3,0.7222222222222222,INLINE_VARIABLE
1,/* * Licensed to the Apache Software Foundatio...,class,14,40,14,106,16,13,0,5,...,0,334,23,26,20,3,0,3,0.6956521739130435,RENAME_PARAMETER
2,/* * Licensed to the Apache Software Foundatio...,class,14,40,14,106,16,13,0,5,...,0,334,23,26,20,3,0,3,0.6956521739130435,RENAME_PARAMETER
3,/* * Licensed to the Apache Software Foundatio...,class,14,40,14,106,16,13,0,5,...,0,334,23,26,20,3,0,3,0.6956521739130435,RENAME_PARAMETER
4,/* * Licensed to the Apache Software Foundatio...,class,14,40,14,92,16,13,0,5,...,0,334,22,25,19,3,0,3,0.6818181818181818,RENAME_PARAMETER


In [20]:
df.to_pickle('../data/instances.pkl')

In [21]:
df['source_code'][0]

'/* * Licensed to the Apache Software Foundation ( ASF ) under one or more * contributor license agreements . See the NOTICE file distributed with * this work for additional information regarding copyright ownership . * The ASF licenses this file to You under the Apache License , Version 2 . 0 * ( the " License "); you may not use this file except in compliance with * the License . You may obtain a copy of the License at * * http :// www . apache . org / licenses / LICENSE - 2 . 0 * * Unless required by applicable law or agreed to in writing , software * distributed under the License is distributed on an " AS IS " BASIS , * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND , either express or implied . * See the License for the specific language governing permissions and * limitations under the License . */  package org . apache . commons . codec . language ;  import java . util . Locale ;  import org . apache . commons . codec . EncoderException ; import org . apache . commons . codec . St